In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
from typing import List, Tuple, Iterable, Dict
from collections import namedtuple, defaultdict

In [2]:
def to_df(grid):
    return pd.DataFrame(grid, columns=['x', 'y', 'z'])

In [3]:
# fig = px.scatter_3d(pd.concat([df, df_2, df_3, zero]), x='x', y='y', z='z', color='beacon')
# fig.show()

In [4]:
class Vector3(namedtuple('Coord', ('x', 'y', 'z'))):
    def rotate24(self):
        """Disclaimer. I got real tired of this part and found a solution online.
        From what I can tell from the thread, nobody liked this part of the problem.
        """
        yield self.__class__((self.x), (self.y), (self.z))
        yield self.__class__((self.y), (self.z), (self.x))
        yield self.__class__((self.z), (self.x), (self.y))
        yield self.__class__((-self.x), (-self.y), (self.z))
        yield self.__class__((-self.y), (-self.z), (self.x))
        yield self.__class__((-self.z), (-self.x), (self.y))
        yield self.__class__((-self.x), (self.y), (-self.z))
        yield self.__class__((-self.y), (self.z), (-self.x))
        yield self.__class__((-self.z), (self.x), (-self.y))
        yield self.__class__((self.x), (-self.y), (-self.z))
        yield self.__class__((self.y), (-self.z), (-self.x))
        yield self.__class__((self.z), (-self.x), (-self.y))
        yield self.__class__((self.y), (self.x), (-self.z))
        yield self.__class__((self.x), (-self.z), (self.y))
        yield self.__class__((-self.z), (self.y), (self.x))
        yield self.__class__((-self.y), (-self.x), (-self.z))
        yield self.__class__((-self.x), (self.z), (self.y))
        yield self.__class__((self.z), (-self.y), (self.x))
        yield self.__class__((-self.y), (self.x), (self.z))
        yield self.__class__((-self.x), (-self.z), (-self.y))
        yield self.__class__((self.z), (self.y), (-self.x))
        yield self.__class__((self.y), (-self.x), (self.z))
        yield self.__class__((self.x), (self.z), (-self.y))
        yield self.__class__((-self.z), (-self.y), (-self.x))
        
    def __add__(self, other: 'Vector3'):
        return self.__class__(self.x + other.x, self.y + other.y, self.z + other.z) 
    
    def __sub__(self, other: 'Vector3'):
        return self.__class__(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def distance_sq(self):
        return self.x * self.x + self.y * self.y + self.z * self.z
    
    def manhattan(self):
        return abs(self.x) + abs(self.y) + abs(self.z)

In [5]:
def rotate24_grid(beacons: List[Vector3]) -> Iterable[List[Vector3]]:
    rotate_iter = [c.rotate24() for c in beacons]
    for i in range(24):
        yield [next(generator) for generator in rotate_iter]
        

def rotate24_grid_to(beacons: List[Vector3], alignment_id: int) -> List[Vector3]:
    beacons_gen = rotate24_grid(beacons)
    for i in range(alignment_id + 1):
        rotated_scanner = next(beacons_gen)
    return rotated_scanner


def translate_grid_to(beacons: List[Vector3], delta: Vector3) -> List[Vector3]:
    return [
        b + delta
        for b in beacons
    ]
        

def parse_scanner(lines: str) -> List[Vector3]:
    return [
        Vector3(*(int(i.strip()) for i in x.split(',') if i.strip()))
        for x in lines.split('\n') if x.strip()
    ]

In [6]:
Fingerprint = Dict[int, Tuple[int, int]]  # distanceSq -> [beaconIndex, beaconIndex]

def fingerprint_scanner(beacons: List[Vector3]) -> Fingerprint:
    """The idea here is to make a set of distances from known nodes.
    sqrt(x^2 + y^2 + z^2) will be unique regardless of orientation.
    Also, we don't need to apply the costly square-root.
    
    A fingerprint should only have UNIQUE distances. If a distance 
    is not unique to the beacon, it is not an identity.
    """
    fingerprint = {}
    duplicate_distances = set()
    for i, coord_i in enumerate(beacons):
        for j, coord_j in enumerate(beacons):
            if i == j:
                continue
            dist = (coord_i - coord_j).distance_sq()
            if dist in duplicate_distances:
                continue
            if dist in fingerprint:
                duplicate_distances.add(dist)
                del fingerprint[dist]
            fingerprint[dist] = (i, j)
    return fingerprint

In [7]:
example_scanners = '''
--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7

--- scanner 0 ---
1,-1,1
2,-2,2
3,-3,3
2,-1,3
-5,4,-6
-8,-7,0

--- scanner 0 ---
-1,-1,-1
-2,-2,-2
-3,-3,-3
-1,-3,-2
4,6,5
-7,0,8

--- scanner 0 ---
1,1,-1
2,2,-2
3,3,-3
1,3,-2
-4,-6,5
7,0,8

--- scanner 0 ---
1,1,1
2,2,2
3,3,3
3,1,2
-6,-4,-5
0,7,-8
'''

# All of these should be equal, because they're in the same orientation
for scanner_lines in example_scanners.split('--- scanner 0 ---'):
    if scanner_lines.strip():
        print(fingerprint_scanner(parse_scanner(scanner_lines)))

{3: (1, 0), 12: (2, 0), 5: (2, 3), 2: (3, 1), 110: (4, 0), 149: (4, 1), 194: (4, 2), 155: (4, 3), 118: (5, 0), 129: (5, 1), 146: (5, 2), 145: (5, 3), 166: (5, 4)}
{3: (1, 0), 12: (2, 0), 5: (2, 3), 2: (3, 1), 110: (4, 0), 149: (4, 1), 194: (4, 2), 155: (4, 3), 118: (5, 0), 129: (5, 1), 146: (5, 2), 145: (5, 3), 166: (5, 4)}
{3: (1, 0), 12: (2, 0), 5: (2, 3), 2: (3, 1), 110: (4, 0), 149: (4, 1), 194: (4, 2), 155: (4, 3), 118: (5, 0), 129: (5, 1), 146: (5, 2), 145: (5, 3), 166: (5, 4)}
{3: (1, 0), 12: (2, 0), 5: (2, 3), 2: (3, 1), 110: (4, 0), 149: (4, 1), 194: (4, 2), 155: (4, 3), 118: (5, 0), 129: (5, 1), 146: (5, 2), 145: (5, 3), 166: (5, 4)}
{3: (1, 0), 12: (2, 0), 5: (2, 3), 2: (3, 1), 110: (4, 0), 149: (4, 1), 194: (4, 2), 155: (4, 3), 118: (5, 0), 129: (5, 1), 146: (5, 2), 145: (5, 3), 166: (5, 4)}


In [8]:
def parse_multiple_scanners(scanners_str: str):
    scanners = []
    curr_scanner = []
    for line in scanners_str.split('\n'):
        if line.startswith('--- scanner'):
            if curr_scanner:
                scanners.append(curr_scanner)            
            curr_scanner = []
        elif line.strip():
            curr_scanner.append(Vector3(*(int(i) for i in line.split(','))))
    scanners.append(curr_scanner)
    return scanners

In [9]:
def get_alignment_ids(v0: Vector3, v1: Vector3):
    """Rotate v1 to match v0. Return a number between 0 and 23 representing the alignment id"""
    for i, v1_rotated in enumerate(v1.rotate24()):
        if v0 == v1_rotated:
            yield i
    return -1


In [10]:
def find_best_rotation(
        scanner_from: List[Vector3], 
        scanner_to: List[Vector3], 
        fingerprint_from: Fingerprint, 
        fingerprint_to: Fingerprint):
    alignment_id_counts = defaultdict(int)
    matching_fingerprints = fingerprint_from.keys() & fingerprint_to.keys()
    for matching_fingerprint in matching_fingerprints:
        i_from, j_from = fingerprint_from[matching_fingerprint]
        i_to, j_to = fingerprint_to[matching_fingerprint]
        x = Vector3(-447,-329,318)
        delta_from = scanner_from[j_from] - scanner_from[i_from]
        delta_to = scanner_to[j_to] - scanner_to[i_to]
        for alignment_id in get_alignment_ids(delta_from, delta_to):
            alignment_id_counts[alignment_id] += 1

    if not alignment_id_counts:
        return None
    return max(alignment_id_counts, key=alignment_id_counts.get)

def find_best_translation(
        scanner_from: List[Vector3], 
        scanner_to: List[Vector3], 
        fingerprint_from: Fingerprint, 
        fingerprint_to: Fingerprint):
    matching_deltas = defaultdict(int)

    for matching_fingerprint in fingerprint_from.keys() & fingerprint_to.keys():
        i_from, j_from = fingerprint_from[matching_fingerprint]
        i_to, j_to = fingerprint_to[matching_fingerprint]
        vfrom_start, vfrom_end = sorted((scanner_from[i_from], scanner_from[j_from]))
        vto_start, vto_end = sorted((scanner_to[i_to], scanner_to[j_to]))
        start_delta = vfrom_start - vto_start
        end_delta = vfrom_end - vto_end
        if start_delta == end_delta:
            matching_deltas[start_delta] += 1
    return max(matching_deltas, key=matching_deltas.get)

### Example

In [18]:
with open('./day19_example_input.txt') as f:
    all_scanners = parse_multiple_scanners(f.read())

scanner0 = all_scanners[0]
translations = []
for scanner1 in all_scanners[1:]:
    fingerprint0 = fingerprint_scanner(scanner0)
    fingerprint1 = fingerprint_scanner(scanner1)
    best_rotation = find_best_rotation(scanner0, scanner1, fingerprint0, fingerprint1)
    rotated_scanner1 = rotate24_grid_to(scanner1, best_rotation)
    best_translation = find_best_translation(scanner0, rotated_scanner1, fingerprint0, fingerprint1)
    translations.append(best_translation)
    translated_scanner1 = translate_grid_to(rotated_scanner1, best_translation)
    print(best_translation)
    scanner0 = list(set(scanner0) | set(translated_scanner1))
    
len(scanner0)

Vector3(x=68, y=-1246, z=-43)
Vector3(x=1105, y=-1205, z=1229)
Vector3(x=-92, y=-2380, z=-20)
Vector3(x=-20, y=-1133, z=1061)


79

In [19]:
expected = set([Vector3(-892,524,684), Vector3(-876,649,763), Vector3(-838,591,734), Vector3(-789,900,-551), Vector3(-739,-1745,668), Vector3(-706,-3180,-659), Vector3(-697,-3072,-689), Vector3(-689,845,-530), Vector3(-687,-1600,576), Vector3(-661,-816,-575), Vector3(-654,-3158,-753), Vector3(-635,-1737,486), Vector3(-631,-672,1502), Vector3(-624,-1620,1868), Vector3(-620,-3212,371), Vector3(-618,-824,-621), Vector3(-612,-1695,1788), Vector3(-601,-1648,-643), Vector3(-584,868,-557), Vector3(-537,-823,-458), Vector3(-532,-1715,1894), Vector3(-518,-1681,-600), Vector3(-499,-1607,-770), Vector3(-485,-357,347), Vector3(-470,-3283,303), Vector3(-456,-621,1527), Vector3(-447,-329,318), Vector3(-430,-3130,366), Vector3(-413,-627,1469), Vector3(-345,-311,381), Vector3(-36,-1284,1171), Vector3(-27,-1108,-65), Vector3(7,-33,-71), Vector3(12,-2351,-103), Vector3(26,-1119,1091), Vector3(346,-2985,342), Vector3(366,-3059,397), Vector3(377,-2827,367), Vector3(390,-675,-793), Vector3(396,-1931,-563), Vector3(404,-588,-901), Vector3(408,-1815,803), Vector3(423,-701,434), Vector3(432,-2009,850), Vector3(443,580,662), Vector3(455,729,728), Vector3(456,-540,1869), Vector3(459,-707,401), Vector3(465,-695,1988), Vector3(474,580,667), Vector3(496,-1584,1900), Vector3(497,-1838,-617), Vector3(527,-524,1933), Vector3(528,-643,409), Vector3(534,-1912,768), Vector3(544,-627,-890), Vector3(553,345,-567), Vector3(564,392,-477), Vector3(568,-2007,-577), Vector3(605,-1665,1952), Vector3(612,-1593,1893), Vector3(630,319,-379), Vector3(686,-3108,-505), Vector3(776,-3184,-501), Vector3(846,-3110,-434), Vector3(1135,-1161,1235), Vector3(1243,-1093,1063), Vector3(1660,-552,429), Vector3(1693,-557,386), Vector3(1735,-437,1738), Vector3(1749,-1800,1813), Vector3(1772,-405,1572), Vector3(1776,-675,371), Vector3(1779,-442,1789), Vector3(1780,-1548,337), Vector3(1786,-1538,337), Vector3(1847,-1591,415), Vector3(1889,-1729,1762), Vector3(1994,-1805,1792)])
actual = set(scanner0)
len(actual & expected), len(actual - expected), len(expected - actual)

(79, 0, 0)

In [20]:
max([
    (j - i).manhattan()
    for i in translations 
    for j in translations
])

3621

### Part 1

In [13]:
with open('./day19_input.txt') as f:
    all_scanners = parse_multiple_scanners(f.read())

translations = []
scanner0 = all_scanners[0]
while all_scanners:
    scanner1 = all_scanners.pop()
    fingerprint0 = fingerprint_scanner(scanner0)
    fingerprint1 = fingerprint_scanner(scanner1)
    best_rotation = find_best_rotation(scanner0, scanner1, fingerprint0, fingerprint1)
    if best_rotation is None:
        all_scanners.insert(0, scanner1)
        continue
    rotated_scanner1 = rotate24_grid_to(scanner1, best_rotation)
    best_translation = find_best_translation(scanner0, rotated_scanner1, fingerprint0, fingerprint1)
    translated_scanner1 = translate_grid_to(rotated_scanner1, best_translation)
    translations.append(best_translation)
    scanner0 = list(set(scanner0) | set(translated_scanner1))
    
len(scanner0)

315

In [15]:
max([
    (j - i).manhattan()
    for i in translations 
    for j in translations
])

13192